# Methods for improving Q2 estiamtes

We have seen that the multiples step training procedure introduces significant errors in the upper atmosphere. These result from the scheme trying to undo the errors it makes in early time steps. I believe this is mainly a result of the humidity in the upper atmosphere being forced to zero

## Weight first prediction step more
![](../models/116/q2_7-8.png)

This helps somewhat but there are still large errors in the upper atmosphere. **All of the following techniques include this**

## Multiply negative humidity by 100 in the loss function
![](../models/119/q2_9-8.png)

This appears to introduce positive bias everywhere, but it is still helpful.


## Humidity loss is $q^{1/4}$
![](../models/120/q2_3-8.png)

Changing the humidity loss function to $(q^{1/4}-q_{true}^{.25}$ does reduce errors in the upper atmosphere, at the cost of errors in the lower atmosphere. Also, this solution blows up.

## Predict $q_T f(x)$

Computing the product rather than the sum helps ensure that there are fewer negative moisture points. Unforunately, the strong drying at the tropopause that we see in many other cases is also present. Perhaps, I should also compute FQT using log.

![](../models/123/q2_9-8.png)

## Adding random noise after every time step

This can be viewed as a form of regularization. For now, I am adding normal random noise with a standard deviation of 5 [units]/day to both SLI and QT at all heights.

![](../models/124/q2_9-8.png)

The loss function is significantly worse when using this form of regularization, which is what we want I think.
Ultimately, even with random noise the solutions seem to converge to the strong drying at the tropopause.

### Increase the noise to 10

![](../models/124/q2_4-8.png)

### Random noise + q * f

Let's try adding random noise along with the Q * F prediction.

![](../models/126/q2_9-8.png)

I think this is best approach I have tried so far.

#### Try with seq_length=10

Now, that I have tried it with seq_length=3, let's try it with seq_length=10.

![](../models/127/q2_2-8.png)

With longer prediction interval this method seems to underfit dramatically.

#### Same as bove but smaller noise.
RunID=129
Make the noise 1.0...I can't make this stable for whatever reason. No decay with time.

#### seq_length=10, noise size=1, no decay

Very strong bias in upper atmos.. **no good**

## Add penalty to loss function

Also penalize the error in Q2 made between the schemes. There are a couple ways to do this:

1. penalize the Q2 computed for the full predicted time series  $\tilde{x}$ (this approach doesn't work well. No pictures unforunately!


## Different loss decay schedule (Runid = 132)

$loss = \sum_i c_i loss_i $. $c = [10.0, r^i,...]$. The first point has ten times the loss of the next point, and then it decays at a constant rate.

![](../models/132/q2_5-8.png)

In [ ]:
from src.sacred import get_run

get_run(132)['config']

This was run with a sequence length of 10. Let's see how a single column model simulation does.

In [ ]:
import torch
from uwnet.columns import single_column_simulation

In [ ]:
ds = xr.open_dataset("../data/processed/training.nc")

def scm_plot(path):
    model = torch.load(path)
    location = ds.isel(x=slice(0,1), y=slice(32,33))
    scm_data = single_column_simulation(model, location, interval=(0, 190))
#     scm_data.QT.squeeze().T.plot.contourf()
    return scm_data, model

In [ ]:
def water_budget_plots(path):
    scm_data, model = scm_plot(path)
    merged_pred_data = location.rename({'SLI': 'SLIOBS', 'QT': 'QTOBS'}).merge(scm_data, join='inner')
    output = model.call_with_xr(merged_pred_data)

    plt.figure()
    output.QT.plot(x='time')
    plt.title("FQTNN from NN-predicted OBS")
    output_truth = model.call_with_xr(location)

    plt.figure()
    output_truth.QT.plot(x='time')
    plt.title("FQTNN from Truth")
    plt.xlim([100, 125])


    plt.figure()
    (scm_data.QT * ds.layer_mass/1000).sum('z').plot()
    (location.QT * ds.layer_mass/1000).sum('z').plot(label='observed')
    plt.legend()

In [ ]:
water_budget_plots("../models/132/4.pkl")

The problem is that there is a long term drift in the humidity in the single column simulation.

## Decay loss with straight r^k

This seems to worsen the results. The solutions oscillate between too much moistening and too much drying

### Epoch 5

In [ ]:
water_budget_plots("../models/133/5.pkl")

### Epoch 6

In [ ]:
water_budget_plots("../models/133/6.pkl")

### Epoch 8

In [ ]:
water_budget_plots("../models/133/8.pkl")